# CSE688: Practical and Application of Machine Learning - Spring 2021
## Assignment 3-2
### Authors

- B073040018 朱劭璿
- B072010029 陳居廷

### References
https://www.tensorflow.org/tutorials/text/classify_text_with_bert

#### Sentiment analysis on the sentences in the attachment `hw3-2-dataset`. Classify the sentences into two categories: possitive (1) amd negative (0)

## Preprocess The Dataset

In [1]:
import tensorflow as tf
import numpy as np

DATA_DIR = '../hw3-2-dataset/'
train_label = []
train_data = []
with open(DATA_DIR+'training.txt', 'r') as f:
    lines = f.readlines()
for l in lines:
    l = l[:-1].split('\t')
    train_label.append(int(l[0]))
    train_data.append(l[1])
    
train_label = np.array(train_label)
train_data = np.array(train_data)

ds_train = tf.data.Dataset.from_tensor_slices((train_data, train_label))
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(train_label.shape[0])
ds_train = ds_train.batch(32)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

print('Label:', train_label[0])
print('Sentence:', train_data[0])
print('Num samples":', train_label.shape[0])

Label: 1
Sentence: The Da Vinci Code book is just awesome.
Num samples": 7086


## Load The Pretrained Model

In [2]:
import tensorflow_hub as hub
import tensorflow_text as text

tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [3]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
    return tf.keras.Model(text_input, net)

classifier_model = build_classifier_model()
classifier_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
preprocessing (KerasLayer)      {'input_type_ids': ( 0           text[0][0]                       
__________________________________________________________________________________________________
BERT_encoder (KerasLayer)       {'default': (None, 5 28763649    preprocessing[0][0]              
                                                                 preprocessing[0][1]              
                                                                 preprocessing[0][2]              
_______________________________________________________________________________________

## Define Hyper Parameters

In [4]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

init_lr = 3e-5
optimizer = tf.keras.optimizers.Adam(init_lr)

classifier_model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics
)

## Training

In [7]:
epochs = 2
classifier_model.fit(
    ds_train,
    epochs=epochs
)

Epoch 1/2
222/222 [==============================] - 29s 130ms/step - loss: 0.0047 - binary_accuracy: 0.9983
Epoch 2/2
222/222 [==============================] - 29s 131ms/step - loss: 0.0018 - binary_accuracy: 0.9996


## Inference

In [8]:
test_data = []
with open(DATA_DIR+'testdata.txt', 'r') as f:
    lines = f.readlines()
for l in lines:
    test_data.append(l[:-1])

predicted = []
for i in range(0, len(test_data), 256):
    predicted.extend(classifier_model(tf.convert_to_tensor(test_data[i:i+256])).numpy().tolist())

assert len(test_data) == len(predicted)
with open('./result.txt', 'w') as f:
    for i in predicted:
        f.write(f'{int(i[0] > 0)}\n')